## 1. Extract Table from Wikipedia Pages


In [0]:
import pandas as pd

In [2]:
dfs=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',match='Postal Code',na_values='Not assigned',keep_default_na=False)
df = dfs[0]
df=df.dropna()
df

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [3]:
df.shape

(103, 3)

## 2. Download Geospatial Data and Join with previous Dataframe

In [4]:
!wget -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data

--2020-06-15 21:20:04--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 119.81.168.75, 161.202.50.39, 119.81.168.76
Connecting to cocl.us (cocl.us)|119.81.168.75|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-06-15 21:20:04--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|119.81.168.75|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-06-15 21:20:06--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 103.116.4.197
Connecting to ibm.box.com (ibm.box.com)|103.116.4.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-06-15 21:20:07--  https://ibm.box.com/public/st

In [5]:
geodata=pd.read_csv('/content/Geospatial_Coordinates.csv')
geodata

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [6]:
df = pd.merge(df,geodata,how='inner',on='Postal Code')
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## 3. Create Analysis and Clustering in Central Toronto

### Import Library

In [0]:
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import folium 

### Specify Area

In [8]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Create Toronto Map

In [9]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [10]:
ctoronto_data = df[df['Borough'] == 'Central Toronto'].reset_index(drop=True)
ctoronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307
4,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678


In [11]:
address = 'Central Toronto, ON'

geolocator = Nominatim(user_agent="ctoronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.6534817, -79.3839347.


Create Map and adding Marker in Toronto Map

In [12]:

map_ctoronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(ctoronto_data['Latitude'], ctoronto_data['Longitude'], ctoronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ctoronto)  
    
map_ctoronto

### Credentials

In [0]:
#To avoid unwanted use of ID and secret output is ommited 
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [0]:
radius = 1500
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [15]:
ctoronto_venues = getNearbyVenues(names=ctoronto_data['Neighborhood'],
                                   latitudes=ctoronto_data['Latitude'],
                                   longitudes=ctoronto_data['Longitude']
                                  )



Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park


Double-click __here__ for the solution.
<!-- The correct answer is:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
--> 

#### Let's check the size of the resulting dataframe

In [16]:
print(ctoronto_venues.shape)
ctoronto_venues.head()

(110, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Roselawn,43.711695,-79.416936,Ceiling Champions,43.713891,-79.420702,Home Service
4,Roselawn,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden


Let's check how many venues were returned for each neighborhood

In [17]:
ctoronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,31,31,31,31,31,31
Davisville North,9,9,9,9,9,9
"Forest Hill North & West, Forest Hill Road Park",4,4,4,4,4,4
Lawrence Park,3,3,3,3,3,3
"Moore Park, Summerhill East",2,2,2,2,2,2
"North Toronto West, Lawrence Park",19,19,19,19,19,19
Roselawn,3,3,3,3,3,3
"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",17,17,17,17,17,17
"The Annex, North Midtown, Yorkville",22,22,22,22,22,22


#### Let's find out how many unique categories can be curated from all the returned venues

In [20]:
print('There are {} uniques categories.'.format(len(ctoronto_venues['Venue Category'].unique())))

There are 61 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [21]:
# one hot encoding
ctoronto_onehot = pd.get_dummies(ctoronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ctoronto_onehot['Neighborhood'] = ctoronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ctoronto_onehot.columns[-1]] + list(ctoronto_onehot.columns[:-1])
ctoronto_onehot = ctoronto_onehot[fixed_columns]

ctoronto_onehot.head()

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Department Store,Dessert Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Garden,Gas Station,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,History Museum,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Park,Pharmacy,Pizza Place,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Roselawn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Roselawn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [23]:
ctoronto_grouped = ctoronto_onehot.groupby('Neighborhood').mean().reset_index()
ctoronto_grouped

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Department Store,Dessert Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Garden,Gas Station,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,History Museum,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Park,Pharmacy,Pizza Place,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.064516,0.000000,0.000000,0.064516,0.000000,0.000000,0.096774,0.032258,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.032258,0.032258,0.032258,0.000000,0.064516,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.064516,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.032258,0.064516,0.000000,0.000000,0.032258,0.000000,0.096774,0.032258,0.000000,0.000000,0.000000,0.000000,0.064516,0.000000,0.032258,0.032258,0.00,0.000000,0.000000,0.000000
1,Davisville North,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.111111,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2,"Forest Hill North & West, Forest Hill Road Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000
3,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
4,"Moore Park, Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
5,"North Toronto West, Lawrence Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.052632,0.157895,0.105263,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0

#### Let's print each neighborhood along with the top 5 most common venues

In [25]:
num_top_venues = 5

for hood in ctoronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ctoronto_grouped[ctoronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0        Dessert Shop  0.10
1      Sandwich Place  0.10
2  Italian Restaurant  0.06
3                 Gym  0.06
4                Café  0.06


----Davisville North----
                  venue  freq
0                 Hotel  0.11
1      Department Store  0.11
2  Gym / Fitness Center  0.11
3                   Gym  0.11
4                  Park  0.11


----Forest Hill North & West, Forest Hill Road Park----
                 venue  freq
0        Jewelry Store  0.25
1                Trail  0.25
2   Mexican Restaurant  0.25
3     Sushi Restaurant  0.25
4  American Restaurant  0.00


----Lawrence Park----
                 venue  freq
0             Bus Line  0.33
1          Swim School  0.33
2                 Park  0.33
3  American Restaurant  0.00
4           Restaurant  0.00


----Moore Park, Summerhill East----
                 venue  freq
0                  Gym   0.5
1                 Park   0.5
2  American Restaurant   0.0
3           Restaurant

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [27]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ctoronto_grouped['Neighborhood']

for ind in np.arange(ctoronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ctoronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Dessert Shop,Sandwich Place,Pizza Place,Gym,Italian Restaurant,Sushi Restaurant,Coffee Shop,Café,Gourmet Shop,Gas Station
1,Davisville North,Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line
2,"Forest Hill North & West, Forest Hill Road Park",Jewelry Store,Trail,Sushi Restaurant,Mexican Restaurant,Fried Chicken Joint,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Yoga Studio
3,Lawrence Park,Swim School,Bus Line,Park,Yoga Studio,Diner,Gym,Grocery Store,Greek Restaurant,Gourmet Shop,Gas Station
4,"Moore Park, Summerhill East",Gym,Park,Yoga Studio,Diner,Gym / Fitness Center,Grocery Store,Greek Restaurant,Gourmet Shop,Gas Station,Garden


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [28]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

ctoronto_grouped_clustering = ctoronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ctoronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 4, 0, 2, 1, 3, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ctoronto_merged = ctoronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ctoronto_merged = ctoronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ctoronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Swim School,Bus Line,Park,Yoga Studio,Diner,Gym,Grocery Store,Greek Restaurant,Gourmet Shop,Gas Station
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936,3,Ice Cream Shop,Garden,Home Service,Yoga Studio,Diner,Gym / Fitness Center,Gym,Grocery Store,Greek Restaurant,Gourmet Shop
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,4,Jewelry Store,Trail,Sushi Restaurant,Mexican Restaurant,Fried Chicken Joint,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Yoga Studio
4,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,1,Clothing Store,Coffee Shop,Chinese Restaurant,Gym / Fitness Center,Fast Food Restaurant,Diner,Mexican Restaurant,Miscellaneous Shop,Park,Rental Car Location


Finally, let's visualize the resulting clusters

In [32]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ctoronto_merged['Latitude'], ctoronto_merged['Longitude'], ctoronto_merged['Neighborhood'], ctoronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

#### Cluster 1

In [34]:
ctoronto_merged.loc[ctoronto_merged['Cluster Labels'] == 0, ctoronto_merged.columns[[1] + list(range(5, ctoronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0,Swim School,Bus Line,Park,Yoga Studio,Diner,Gym,Grocery Store,Greek Restaurant,Gourmet Shop,Gas Station


#### Cluster 2

In [35]:
ctoronto_merged.loc[ctoronto_merged['Cluster Labels'] == 1, ctoronto_merged.columns[[1] + list(range(5, ctoronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Central Toronto,1,Hotel,Sandwich Place,Gym,Food & Drink Shop,Park,Pizza Place,Department Store,Gym / Fitness Center,Breakfast Spot,Bus Line
4,Central Toronto,1,Clothing Store,Coffee Shop,Chinese Restaurant,Gym / Fitness Center,Fast Food Restaurant,Diner,Mexican Restaurant,Miscellaneous Shop,Park,Rental Car Location
5,Central Toronto,1,Sandwich Place,Café,Coffee Shop,Pizza Place,History Museum,Indian Restaurant,Donut Shop,Liquor Store,Middle Eastern Restaurant,Park
6,Central Toronto,1,Dessert Shop,Sandwich Place,Pizza Place,Gym,Italian Restaurant,Sushi Restaurant,Coffee Shop,Café,Gourmet Shop,Gas Station
8,Central Toronto,1,Pub,Light Rail Station,Coffee Shop,Sports Bar,Vietnamese Restaurant,Fried Chicken Joint,Liquor Store,Pizza Place,Restaurant,American Restaurant


#### Cluster 3

In [36]:
ctoronto_merged.loc[ctoronto_merged['Cluster Labels'] == 2, ctoronto_merged.columns[[1] + list(range(5, ctoronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Central Toronto,2,Gym,Park,Yoga Studio,Diner,Gym / Fitness Center,Grocery Store,Greek Restaurant,Gourmet Shop,Gas Station,Garden


#### Cluster 4

In [38]:
ctoronto_merged.loc[ctoronto_merged['Cluster Labels'] == 3, ctoronto_merged.columns[[1] + list(range(5, ctoronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Central Toronto,3,Ice Cream Shop,Garden,Home Service,Yoga Studio,Diner,Gym / Fitness Center,Gym,Grocery Store,Greek Restaurant,Gourmet Shop


#### Cluster 5

In [40]:
ctoronto_merged.loc[ctoronto_merged['Cluster Labels'] == 4, ctoronto_merged.columns[[1] + list(range(5, ctoronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Central Toronto,4,Jewelry Store,Trail,Sushi Restaurant,Mexican Restaurant,Fried Chicken Joint,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Yoga Studio
